# 03. A/B Test

### 1. 가설 설정 (Hypothesis Setting)
- **귀무가설(H0)**: 두 그룹의 간 전환율 차이는 없다.
- **대립가설(H1)**: 두 그룹의 간 전환율 차이는 있다.

### 2. 통계 검정 대상 및 방법 (Test Design)
1. **광고 효과 전체**
    - 변수: `test group` ↔ `converted`
    - 검정: 카이제곱 검정 (범주형-범주형 변수)
<br><br>
2. **광고 효과의 세부 요인**
    - 요일 효과: `test group` + `most ads day` ↔ `converted`
    - 시간대 효과: `test group` + `most ads hour` ↔ `converted`
    - 노출량 효과: `test group` + `total ads` ↔ `converted`
         
    → 카이제곱 검정 또는 로지스틱 회귀 활용

### 3. 검정 절차 (Step by Step)
> 1. 데이터 분할 및 집계
> 2. 상황별 통계 검정 수행 (카이제곱 / Z-test / 회귀 등)
> 3. p-value 계산
> 4. 유의수준(α = 0.05)과 비교하여 가설 기각 여부 판단
> 5. 효과 크기(전환율 차이, 오즈비 등) 산출

### 4. 결과 해석 (Interpretation)
- 광고가 전환율에 유의미한 영향을 미치는가?
- 어느 요인이 전환율 차이를 크게 만드는가? (요일, 시간대, 노출량)
- 단순 효과인지, 교차 효과(상호작용)까지 확인할 필요가 있는가?

### 5. 결론 (Conlusion)
- 실험 결과 종합 요약
- 광고 전략적 시사점 도출

In [10]:
import pandas as pd
import scipy.stats as stats

# 데이터 불러오기
df = pd.read_csv("../dataset/marketing_AB.csv")

# 광고 그룹(ad) vs PSA 그룹(psa) 전환율 분할표
contingency = pd.crosstab(df['test group'], df['converted'])
print(contingency, "\n")

# 1. 카이제곱 검정 (광고 전체 효과)
chi2, p, dof, expected = stats.chi2_contingency(contingency)

print(f"Chi2: {chi2:.4f}, p-value: {p:.4e}")

alpha = 0.05
if p < alpha:
    print("→ p < 0.05: 귀무가설 기각, 광고 그룹과 PSA 그룹의 전환율 차이는 통계적으로 유의함")
else:
    print("→ p >= 0.05: 귀무가설 채택, 광고 그룹과 PSA 그룹의 전환율 차이는 통계적으로 유의하지 않음")

converted    False  True 
test group               
ad          550154  14423
psa          23104    420 

Chi2: 54.0058, p-value: 1.9990e-13
→ p < 0.05: 귀무가설 기각, 광고 그룹과 PSA 그룹의 전환율 차이는 통계적으로 유의함


In [15]:
# 2. 요일 효과 (test group + 요일 ↔ converted)
contingency_day = pd.crosstab([df['most ads day'], df['test group']], df['converted'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_day)
print(f"Chi2: {chi2:.4f}, p-value: {p:.4e}")

alpha = 0.05
if p < alpha:
    print("→ p < 0.05: 귀무가설 기각, 요일과 그룹에 따른 전환율 차이는 통계적으로 유의함")
else:
    print("→ p >= 0.05: 귀무가설 채택, 요일과 그룹에 따른 전환율 차이는 통계적으로 유의하지 않음")

Chi2: 481.0886, p-value: 1.5146e-94
→ p < 0.05: 귀무가설 기각, 요일과 그룹에 따른 전환율 차이는 통계적으로 유의함


In [16]:
# 3. 시간대 효과 (test group + 시간 ↔ converted)
contingency_hour = pd.crosstab([df['most ads hour'], df['test group']], df['converted'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_hour)
print(f"Chi2: {chi2:.4f}, p-value: {p:.4e}")

alpha = 0.05
if p < alpha:
    print("→ p < 0.05: 귀무가설 기각, 시간대와 그룹에 따른 전환율 차이는 통계적으로 유의함")
else:
    print("→ p >= 0.05: 귀무가설 채택, 시간대와 그룹에 따른 전환율 차이는 통계적으로 유의하지 않음")

Chi2: 496.7429, p-value: 2.1638e-76
→ p < 0.05: 귀무가설 기각, 시간대와 그룹에 따른 전환율 차이는 통계적으로 유의함


In [19]:
# 4. 노출량 효과 (test_group + 노출량 ↔ converted)
# 노출량(total_ads)은 연속형 변수이므로 로지스틱 회귀를 적용

df = df.rename(columns={'test group': 'test_group', 'total ads': 'total_ads'})

# converted를 0/1 정수로 변환
df['converted'] = df['converted'].astype(int)

# 로지스틱 회귀 실행
import statsmodels.formula.api as smf

model = smf.logit("converted ~ C(test_group) + total_ads", data=df).fit()
print(model.summary())

# 노출량(total_ads) 유의성만 추가 체크
p_ads = model.pvalues["total_ads"]

alpha = 0.05
if p_ads < alpha:
    print("→ p < 0.05: 노출량(total_ads)은 전환율에 유의한 영향을 미침")
else:
    print("→ p >= 0.05: 노출량(total_ads)은 전환율에 유의한 영향을 주지 않음")

Optimization terminated successfully.
         Current function value: 0.108986
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:               588101
Model:                          Logit   Df Residuals:                   588098
Method:                           MLE   Df Model:                            2
Date:                Tue, 26 Aug 2025   Pseudo R-squ.:                 0.07467
Time:                        02:23:29   Log-Likelihood:                -64095.
converged:                       True   LL-Null:                       -69267.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -4.0412      0.010   -398.367      0.000      -4.061      -4.021

In [20]:
import numpy as np

# 계수와 odds ratio 계산
params = model.params
odds_ratios = np.exp(params)

print("\n회귀계수 (log-odds):")
print(params)
print("\n승산비 (odds ratio):")
print(odds_ratios)

# 해석 출력
print("\n해석:")
print(f"- 그룹 효과: PSA 그룹의 전환 확률은 광고 그룹(ad)에 비해 "
      f"{odds_ratios['C(test_group)[T.psa]']:.2f}배 수준")
print(f"- 노출량 효과: 광고 노출(total_ads) 1회 증가할 때마다 전환 확률의 승산이 "
      f"{odds_ratios['total_ads']:.3f}배 곱해짐")


회귀계수 (log-odds):
Intercept              -4.041237
C(test_group)[T.psa]   -0.389428
total_ads               0.010145
dtype: float64

승산비 (odds ratio):
Intercept               0.017576
C(test_group)[T.psa]    0.677444
total_ads               1.010197
dtype: float64

해석:
- 그룹 효과: PSA 그룹의 전환 확률은 광고 그룹(ad)에 비해 0.68배 수준
- 노출량 효과: 광고 노출(total_ads) 1회 증가할 때마다 전환 확률의 승산이 1.010배 곱해짐


### 통계적 가설 검정 요약

#### 1. 광고 효과 유무
- **가설**
    - H0: 광고 그룹과 PSA 그룹의 전환율 차이는 없다.
    - H1: 광고 그룹과 PSA 그룹의 전환율 차이는 있다.
- **결과**
    - 광고 그룹 전환율: 약 **2.55%**
    - PSA 그룹 전환율: 약 **1.78%**
    - p-value < 0.05 → **귀무가설 기각**
- **해석**
    - 광고 그룹의 전환율이 PSA 그룹보다 **유의하게 높음**
 
#### 2. 요일 효과 (카이제곱 검정)
- **가설**
    - H0: 요일에 따른 전환율 차이는 없다.
    - H1: 요일에 따른 전환율 차이는 있다.
- **결과**
    - 요일별 전환율에 차이 존재
    - 특히 **월요일·화요일 전환율이 높음**
    - p-value < 0.05 → **귀무가설 기각**
- **해석**
    - 요일은 전환율에 영향을 주는 요인으로 작용

#### 3. 시간대 효과 (카이제곱 검정)
- **가설**
    - H0: 시간대에 따른 전환율 차이는 없다.
    - H1: 시간대에 따른 전환율 차이는 있다.
- **결과**
    - **15 ~ 17시, 20 ~ 21시** 전환율이 두드러지게 높음
    - p-value < 0.05 → **귀무가설 기각**
- **해석**
    - 시간대별 광고 효과가 다르며, 특정 시간대 집중 광고가 효과적

#### 4. 노출량 효과 (로지스틱 회귀)
- **가설**
    - H0: 노출량은 전환율에 영향을 주지 않는다.
    - H1: 노출량은 전환율에 영향을 준다.
- **결과**
    - total_ads 계수 = +0.01 (p < 0.001)
    - 노출량이 많을수록 전환 확률 증가
- **해석**
    - 광고를 **더 자주 노출할수록 전환율이 상승**하는 경향 확인
 
#### 5. 종합 결론
> 광고 그룹이 PSA 그룹보다 **전환율에서 확실히 우위**
> 전환율은 **요일·시간대별 차이** 존재
> 노출량은 **전환율을 끌어올리는 요인 중 하나**
> 따라서, **광고 집행 전략은 특정 요일·시간대에 집중하고 충분한 노출량 확보**가 핵심이다.

#### 6. 전략적 시사점
1. **예산 최적화** → 광고비를 분산하기보다 **월·화 / 오후 15 ~ 17시, 20 ~ 21시**에 집중 투자
2. **캠페인 운영** → 단순한 광고 집행보다 **충분한 노출 빈도 확보**가 전환율 상승에 기여
3. **광고 집행 효과 검증** → 향후 A/B 테스트는 새로운 광고 소재, 디자인 등 적용 가능